In [1]:
import wibblywobbly as ww

catalog = ["Mouse", "Cat", "Dog", "Human"]
data = ["mice",  "CAT ", "doggo", "PERSON", 999]

In [3]:
ww.map_list_to_catalog(data, catalog)

{999: 999, 'CAT ': 'Cat', 'doggo': 'Dog', 'PERSON': 'PERSON', 'mice': 'mice'}

In [8]:
ww.map_list_to_catalog(data, catalog, thr_reject=40, reject_value='Other')

{999: 999, 'CAT ': 'Cat', 'doggo': 'Dog', 'PERSON': 'Other', 'mice': 'Mouse'}

In [4]:
ww.map_list_to_catalog(data, catalog, warnings=True, thr_warn=95)

WOBBLY: doggo
	Options: Dog (90), Mouse (20), Cat (0)
REJECT: PERSON
	Options: Dog (30), Mouse (18), Human (18)
REJECT: mice
	Options: Mouse (44), Cat (29), Human (22)


{999: 999, 'CAT ': 'Cat', 'doggo': 'Dog', 'PERSON': 'PERSON', 'mice': 'mice'}

In [2]:
ww.map_list_to_catalog(data, catalog, result="dataframe")

,Data,Option1,Score1,Option2,Score2,Option3,Score3
0,CAT,Cat,100,Human,30.0,Dog,0.0
1,doggo,Dog,90,Mouse,20.0,Cat,0.0
2,mice,Mouse,44,Cat,29.0,Human,22.0
3,PERSON,Dog,30,Mouse,18.0,Human,18.0
4,999,999,0,None,NaN,None,NaN
